# DeepRetinotopy Toolbox

**Authors**: Fernanda Ribeiro

**Date**: 12 Nov 2025

### Citation and Resources
#### Tools included in this workflow
__deepretinotopy__:
- Ribeiro, F. L., Bollmann, S., & Puckett, A. M. (2021). Predicting the retinotopic organization of human visual cortex from anatomy using geometric deep learning. NeuroImage, 244, 118624. [https://doi.org/10.1016/j.neuroimage.2021.118624](https://doi.org/10.1016/j.neuroimage.2021.118624)

#### Dataset
__NYU Retinotopy Dataset__

Himmelberg, Marc M. and Kurzawski, Jan W. and Benson, Noah C. and Carrasco, Marisa and Winawer, Jonathan. “NYU Retinotopy dataset” 2021. [doi:10.18112/openneuro.ds003787.v1.0.1](https://github.com/OpenNeuroDatasets/ds003787.git)


## DeepRetinotopy toolbox

This interactive notebook demonstrates how to predict retinotopic maps from anatomical data using a geometric deep learning model of retinotopy. For more details about the toolbox, please check our [repository](https://github.com/felenitaribeiro/deepRetinotopy_Toolbox). The accompanying manuscript will be available soon.

Requirements:

1. freesurfer directory;
2. HCP "fs_LR-deformed_to-fsaverage" surfaces, available [here](https://github.com/Washington-University/HCPpipelines/tree/master/global/templates/standard_mesh_atlases/resample_fsaverage).


## Load Software Tools

In [1]:
%%capture 
!pip install nilearn nibabel numpy

In [2]:
# we can use module to load deepretinotopy in a specific version
import module
import os
await module.load('deepretinotopy/1.0.18')
await module.list()

['deepretinotopy/1.0.18']

In [3]:
# Request a freesurfer license and store it in your homedirectory. This is just an example - please replace with your license id:
!echo "Steffen.Bollmann@cai.uq.edu.au" > ~/.license
!echo "21029" >> ~/.license
!echo "*Cqyn12sqTCxo" >> ~/.license
!echo "FSxgcvGkNR59Y" >> ~/.license

## Download data

Let's start by downloading some data available on OpenNeuro. DeepRetinotopy only requires a minimal subset of FreeSurfer outputs, which means you can significantly reduce storage requirements by downloading only the following files per subject:

### Essential Files
```
<subject_id>/surf/
├── lh.white          # White matter surface
├── lh.pial           # Pial surface  
├── lh.sphere.reg     # Spherical surface for resampling
├── rh.white          # White matter surface
├── rh.pial           # Pial surface
└── rh.sphere.reg     # Spherical surface for resampling
```
**_Note_**: The toolbox automatically generates the midthickness surfaces and curvature data from these base files during processing. All other FreeSurfer outputs (thickness maps, parcellations, etc.) are not required for retinotopic mapping prediction.

In [5]:
!mkdir -p ./data

os.chdir('./data')
!datalad install https://github.com/OpenNeuroDatasets/ds003787.git # install NYU retinotopy dataset
os.chdir('ds003787/derivatives/freesurfer') # change directories
list_of_subs = os.listdir('.')

# download required files to run deepretinotopy
for sub in list_of_subs:
    for hemisphere in ['lh', 'rh']:
        for file in ['pial', 'white', 'sphere', 'sphere.reg', 'thickness']:
            cmd=f"""datalad get {sub}/surf/{hemisphere}.{file}""" 
            os.system(cmd)

Cloning:   0%|                             | 0.00/2.00 [00:00<?, ? candidates/s]
Enumerating: 0.00 Objects [00:00, ? Objects/s]
                                              
Counting:   0%|                               | 0.00/137k [00:00<?, ? Objects/s]
                                                                                
Compressing:   0%|                            | 0.00/113k [00:00<?, ? Objects/s]
                                                                                
Receiving:   0%|                              | 0.00/137k [00:00<?, ? Objects/s]
Receiving:  16%|███▌                  | 21.9k/137k [00:00<00:00, 216k Objects/s]
Receiving:  36%|███████▉              | 49.2k/137k [00:00<00:00, 240k Objects/s]
Receiving:  56%|████████████▎         | 76.6k/137k [00:00<00:00, 254k Objects/s]
Receiving:  75%|█████████████████▎     | 103k/137k [00:00<00:00, 249k Objects/s]
Receiving:  94%|█████████████████████▌ | 129k/137k [00:00<00:00, 246k Objects/s]
               

Since we have preprocessed data available, the only remaining requirements are the HCP template surfaces. 
Thus, we need to create a "templates" folder (for example, ./templates/) and download the following files from the link above:

- "fs_LR-deformed_to-fsaverage.R.sphere.32k_fs_LR.surf.gii";
- "fs_LR-deformed_to-fsaverage.L.sphere.32k_fs_LR.surf.gii";

In [6]:
# Set up paths relative to current directory
os.environ['PATH_FREESURFER_DIR'] = "."
os.environ['PATH_HCP_TEMPLATE_SURFACE'] = "../../templates"
os.environ['DATASET_NAME'] = "nyu"
os.environ['MAPS'] = "polarAngle,eccentricity"

In [7]:
# Create templates directory
os.makedirs(os.environ['PATH_HCP_TEMPLATE_SURFACE'], exist_ok=True)

In [8]:
# Download templates to the relative path
!cd $PATH_HCP_TEMPLATE_SURFACE && \
wget https://github.com/Washington-University/HCPpipelines/raw/master/global/templates/standard_mesh_atlases/resample_fsaverage/fs_LR-deformed_to-fsaverage.L.sphere.32k_fs_LR.surf.gii && \
wget https://github.com/Washington-University/HCPpipelines/raw/master/global/templates/standard_mesh_atlases/resample_fsaverage/fs_LR-deformed_to-fsaverage.R.sphere.32k_fs_LR.surf.gii

--2025-11-12 23:39:46--  https://github.com/Washington-University/HCPpipelines/raw/master/global/templates/standard_mesh_atlases/resample_fsaverage/fs_LR-deformed_to-fsaverage.L.sphere.32k_fs_LR.surf.gii
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Washington-University/HCPpipelines/master/global/templates/standard_mesh_atlases/resample_fsaverage/fs_LR-deformed_to-fsaverage.L.sphere.32k_fs_LR.surf.gii [following]
--2025-11-12 23:39:46--  https://raw.githubusercontent.com/Washington-University/HCPpipelines/master/global/templates/standard_mesh_atlases/resample_fsaverage/fs_LR-deformed_to-fsaverage.L.sphere.32k_fs_LR.surf.gii
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.1

## Running deepRetinotopy for a single subject

Here, we illustrate how to run deepRetinotopy for a single subject. Note that, if no output directory is provided, all maps are saved within each subject subdirectory in the freesurfer directory. 

In [9]:
## Process single subject without an explicit output directory
!deepRetinotopy -s $PATH_FREESURFER_DIR -t $PATH_HCP_TEMPLATE_SURFACE -d $DATASET_NAME -m $MAPS -i sub-wlsubj001

Processing single subject: sub-wlsubj001
Log files location: /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer/logs
Output log: deepRetinotopy_20251112_234014_output.log
Error log: deepRetinotopy_20251112_234014_error.log
--------------------------------------------------------------------------------
DeepRetinotopy: retinotopic mapping using deep learning
--------------------------------------------------------------------------------
Log files: /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer/logs/deepRetinotopy_20251112_234014_output.log and deepRetinotopy_20251112_234014_error.log
--------------------------------------------------------------------------------
Directories and parameters:

Path to FreeSurfer directory: /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer
Path to fs_LR-deformed_to-fsaverage t

In [10]:
!ls $PATH_FREESURFER_DIR/sub-wlsubj001/deepRetinotopy

sub-wlsubj001.ROI.lh.native.label.gii
sub-wlsubj001.ROI.rh.native.label.gii
sub-wlsubj001.fs_predicted_eccentricity_lh_curvatureFeat_model.func.gii
sub-wlsubj001.fs_predicted_eccentricity_rh_curvatureFeat_model.func.gii
sub-wlsubj001.fs_predicted_polarAngle_lh_curvatureFeat_model.func.gii
sub-wlsubj001.fs_predicted_polarAngle_rh_curvatureFeat_model.func.gii
sub-wlsubj001.predicted_eccentricity_model.lh.native.func.gii
sub-wlsubj001.predicted_eccentricity_model.rh.native.func.gii
sub-wlsubj001.predicted_polarAngle_model.lh.native.func.gii
sub-wlsubj001.predicted_polarAngle_model.rh.native.func.gii


In [12]:
## Process single subject with an explicit output directory
! mkdir -p ../../../output && \
deepRetinotopy -s $PATH_FREESURFER_DIR -t $PATH_HCP_TEMPLATE_SURFACE -d $DATASET_NAME -m $MAPS -i sub-wlsubj001 -o ../../../output

Processing single subject: sub-wlsubj001
Log files location: /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/output
Output log: deepRetinotopy_20251113_000605_output.log
Error log: deepRetinotopy_20251113_000605_error.log
--------------------------------------------------------------------------------
DeepRetinotopy: retinotopic mapping using deep learning
--------------------------------------------------------------------------------
Log files: /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/output/deepRetinotopy_20251113_000605_output.log and deepRetinotopy_20251113_000605_error.log
--------------------------------------------------------------------------------
Directories and parameters:

Path to FreeSurfer directory: /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer
Path to fs_LR-deformed_to-fsaverage template surfaces: /home/jovyan/Git_repositories/example-note

In [13]:
!ls ../../../output/sub-wlsubj001/

deepRetinotopy	surf


## Running deepRetinotopy for multiple subjects

If no subject is specified, all subjects within the FreeSurfer directory will be processed. However, for optimal speed, parallelising the processing for multiple subjects across multiple cores is preferred.

In [14]:
## Determine how many cores you have available
os.environ['NUMBER_OF_CORES'] = str(os.cpu_count() - 1)

In [15]:
%%bash

SUBJECT_LIST=(sub-wlsubj001 sub-wlsubj004 sub-wlsubj006 sub-wlsubj007)

process_subject() {
    local subject=$1
    echo "Processing subject: $subject"
    deepRetinotopy -s $PATH_FREESURFER_DIR -t $PATH_HCP_TEMPLATE_SURFACE -d $DATASET_NAME -m $MAPS -i "$subject"
}

export -f process_subject

printf "%s\n" "${SUBJECT_LIST[@]}" | xargs -P "$NUMBER_OF_CORES" -I {} bash -c 'process_subject "{}"'

Processing subject: sub-wlsubj001
Processing subject: sub-wlsubj004
Processing subject: sub-wlsubj006
Processing subject: sub-wlsubj007
Processing single subject: sub-wlsubj004
Processing single subject: sub-wlsubj001
Processing single subject: sub-wlsubj006
Processing single subject: sub-wlsubj007
Log files location: /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer/logs
Output log: deepRetinotopy_20251113_001041_output.log
Error log: deepRetinotopy_20251113_001041_error.log
--------------------------------------------------------------------------------
DeepRetinotopy: retinotopic mapping using deep learning
--------------------------------------------------------------------------------
Log files: /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer/logs/deepRetinotopy_20251113_001041_output.log and deepRetinotopy_20251113_001041_error.log
----------------------------

/opt/miniconda-latest/lib/python3.12/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /usr/lib64/libm.so.6: version `GLIBC_2.29' not found (required by /opt/miniconda-latest/lib/python3.12/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/opt/miniconda-latest/lib/python3.12/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/lib64/libm.so.6: version `GLIBC_2.29' not found (required by /opt/miniconda-latest/lib/python3.12/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


Saving /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer/sub-wlsubj006/surf/lh.pial.gii as a surface
Saving /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer/sub-wlsubj007/surf/lh.pial.gii as a surface
[sub-wlsubj006] Generating midthickness surface...
[sub-wlsubj007] Generating midthickness surface...
Saving /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer/sub-wlsubj004/surf/lh.pial.gii as a surface
[sub-wlsubj004] Generating midthickness surface...
Midthickness surface saved at /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer/sub-wlsubj006/surf/lh.graymid.gii
Midthickness surface saved at /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer/sub-wlsubj007/surf/lh.graymid.gii


** DA[1] has coordsys with intent NIFTI_INTENT_TRIANGLE (should be NIFTI_INTENT_POINTSET)
** DA[1] has coordsys with intent NIFTI_INTENT_TRIANGLE (should be NIFTI_INTENT_POINTSET)
Processing...


Midthickness surface saved at /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer/sub-wlsubj004/surf/lh.graymid.gii


...

/opt/miniconda-latest/lib/python3.12/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/lib64/libm.so.6: version `GLIBC_2.29' not found (required by /opt/miniconda-latest/lib/python3.12/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
/opt/miniconda-latest/lib/python3.12/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/lib64/libm.so.6: version `GLIBC_2.29' not found (required by /opt/miniconda-latest/lib/python3.12/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
/opt/miniconda-latest/lib/python3.12/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /usr/lib64/libm.so.6: version `GLIBC_2.29' not found (required by /opt/minic

[Step 2] STARTING INFERENCE
Prediction type: eccentricity
Hemisphere: rh
Dataset: nyu
Stimulus: original
Output mode: In-place (within FreeSurfer directory structure)
Output directory not specified, using input path: /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer
Data path: /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer
Temporary processed directory: /home/jovyan/Git_repositories/example-notebooks/books/functional_imaging/data/ds003787/derivatives/freesurfer/processed_3030651_925f0cfe_34146
Processing single subject: sub-wlsubj006
Found 1 subjects to process: ['sub-wlsubj006']
Final subject list after validation: 1 subjects: ['sub-wlsubj006']
[Step 2.1] Loading the dataset...
Dataset loaded successfully
Dataset loading time: 0.29 minutes
[Step 2.2] Generating predictions with pre-trained models...
Loading model 1/1...
Using device: cpu
Loading model from: deepRe

[sub-wlsubj007] Applying mask to the predicted map of the right hemisphere...
[sub-wlsubj006] Applying mask to the predicted map of the right hemisphere...


=== Subject sub-wlsubj007 completed in 0 minutes ===

[Step 3] COMPLETED!
Total execution time: 0m 4s
Subject processed: sub-wlsubj007
Map: eccentricity | Model: model | Hemisphere: rh
Output location: In-place within FreeSurfer directory
eccentricity resampling completed for rh hemisphere
--------------------------------------------------------------------------------
Execution completed!
=== Subject sub-wlsubj006 completed in 0 minutes ===

[Step 3] COMPLETED!
Total execution time: 0m 4s
Subject processed: sub-wlsubj006
Map: eccentricity | Model: model | Hemisphere: rh
Output location: In-place within FreeSurfer directory
eccentricity resampling completed for rh hemisphere
--------------------------------------------------------------------------------
Execution completed!
[sub-wlsubj004] Applying mask to the predicted map of the right hemisphere...


=== Subject sub-wlsubj004 completed in 0 minutes ===

[Step 3] COMPLETED!
Total execution time: 0m 4s
Subject processed: sub-wlsubj004
Map: eccentricity | Model: model | Hemisphere: rh
Output location: In-place within FreeSurfer directory
eccentricity resampling completed for rh hemisphere
--------------------------------------------------------------------------------
Execution completed!


In [17]:
!ls ./sub-wlsubj004/deepRetinotopy

sub-wlsubj004.ROI.lh.native.label.gii
sub-wlsubj004.ROI.rh.native.label.gii
sub-wlsubj004.fs_predicted_eccentricity_lh_curvatureFeat_model.func.gii
sub-wlsubj004.fs_predicted_eccentricity_rh_curvatureFeat_model.func.gii
sub-wlsubj004.fs_predicted_polarAngle_lh_curvatureFeat_model.func.gii
sub-wlsubj004.fs_predicted_polarAngle_rh_curvatureFeat_model.func.gii
sub-wlsubj004.predicted_eccentricity_model.lh.native.func.gii
sub-wlsubj004.predicted_eccentricity_model.rh.native.func.gii
sub-wlsubj004.predicted_polarAngle_model.lh.native.func.gii
sub-wlsubj004.predicted_polarAngle_model.rh.native.func.gii


## Maps visualization

Finally, we can visualize the predicted maps in the individual's native space using nilearn.

In [28]:
from nilearn import plotting
from ipywidgets import Dropdown, interact
import nibabel as nib
import numpy as np
from IPython.display import display


def plot_data(subject, retinotopic_map):
    background = np.array(nib.load(f"./{subject}/surf/lh.graymid.H.gii").agg_data())[:]
    background[background < 0] = -1
    background[background > 1] = 1
    threshold = 1
    data = np.array(nib.load(f"./{subject}/deepRetinotopy/{subject}.predicted_{retinotopic_map}_model.lh.native.func.gii").agg_data())[:]
    data[data != 0] = data[data != 0] + threshold
    if retinotopic_map == 'eccentricity':
        max_value = 8
    else: 
        max_value = 360
    view = plotting.view_surf(
        surf_mesh=f"./{subject}/surf/lh.midthickness.surf.gii",
        surf_map=np.reshape(data[:], (-1)),
        cmap="gist_rainbow_r", black_bg=False, symmetric_cmap=False, bg_map = background,
        threshold=threshold, vmax=max_value + threshold, vmin = threshold, darkness=None)
    return view


In [29]:
subjects = ['sub-wlsubj001', 'sub-wlsubj004', 'sub-wlsubj006', 'sub-wlsubj007']
retinotopic_maps = ['eccentricity', 'polarAngle']

@interact(subject = subjects, retinotopic_map = retinotopic_maps)
def plot1(subject, retinotopic_map):
    view = plot_data(subject, retinotopic_map)
    display(view)

interactive(children=(Dropdown(description='subject', options=('sub-wlsubj001', 'sub-wlsubj004', 'sub-wlsubj00…

### Dependencies in Jupyter/Python
- Using the package [watermark](https://github.com/rasbt/watermark) to document system environment and software versions used in this notebook

In [27]:
%load_ext watermark

%watermark
%watermark --iversions

Last updated: 2025-11-13T00:57:44.290157+00:00

Python implementation: CPython
Python version       : 3.11.6
IPython version      : 8.16.1

Compiler    : GCC 12.3.0
OS          : Linux
Release     : 5.4.0-204-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 32
Architecture: 64bit

nilearn   : 0.11.1
ipywidgets: 8.1.2
numpy     : 2.2.4
IPython   : 8.16.1
nibabel   : 5.3.2

